In [ ]:
!pip install --quiet qiskit qiskit-ibm-runtime qiskit-aer qiskit-optimization dotenv qiskit-algorithms

## Set up env and backend

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

from qiskit_ibm_runtime import QiskitRuntimeService

api_key = os.getenv("api_key")
crn = os.getenv("crn")

# connect to backend
QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=api_key,
    instance=crn,
    overwrite=True
)
# Check that the account has been saved properly
service = QiskitRuntimeService()
service.saved_accounts()

# check backends
service.backends()



[<IBMBackend('ibm_torino')>, <IBMBackend('ibm_brisbane')>]

In [ ]:
from qiskit_aer import Aer
sampler = Sampler(Aer.get_backend("aer_simulator"))


In [ ]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver

prob = Knapsack(values=[3,4,5,6,7], weights=[2,3,4,5,6], max_weight=10)
qp = prob.to_quadratic_program()

print(qp.prettyprint())

In [ ]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))

In [ ]:
# from qiskit.primitives import StatevectorSampler

# # QAOA
# meo = MinimumEigenOptimizer(
#     min_eigen_solver=QAOA(reps=1, sampler=StatevectorSampler(seed=123), optimizer=COBYLA())
# )
# result = meo.solve(qp)
# print(result.prettyprint())
# print("\nsolution:", prob.interpret(result))
# print("\ntime:", result.min_eigen_solver_result.optimizer_time)

## Quantum

In [ ]:
backend = service.backend("ibm_brisbane")
print("Using backend:", backend)

In [12]:
import numpy as np
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import efficient_su2

# --- Define knapsack ---
values = [3, 4, 5, 6, 7]
weights = [2, 3, 4, 5, 6]
capacity = 10

prob = Knapsack(values=values, weights=weights, max_weight=capacity)
qp = prob.to_quadratic_program()
print("Quadratic program:")
print(qp.prettyprint())

# --- Backend ---
backend = service.backend("ibm_brisbane")
print("\nUsing backend:", backend)

# --- Sampler ---
sampler = Sampler(mode=backend)

# --- Optimizer ---
optimizer = COBYLA(maxiter=50)

# --- Convert problem to Ising ---
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
op, offset = qubo.to_ising()

# --- Hardware-friendly ansatz ---
num_qubits = op.num_qubits
ansatz = efficient_su2(num_qubits, reps=1, entanglement="linear")

# --- SamplingVQE (let sampler transpile automatically) ---
vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer)

# --- Solve ---
result = vqe.compute_minimum_eigenvalue(op)

# --- Interpret results ---
best_x = None
best_value = -1e9
for sample in result.samples:
    x = [int(b) for b in sample.binary[::-1]]
    value = sum(values[i] * x[i] for i in range(len(values)))
    weight = sum(weights[i] * x[i] for i in range(len(weights)))
    if weight <= capacity and value > best_value:
        best_value, best_x = value, x

print("\nVQE result on hardware:")
print("Eigenvalue:", result.eigenvalue)
if best_x is not None:
    print("Best solution:", best_x)
    print("Value:", best_value, "Weight:", sum(weights[i] * best_x[i] for i in range(len(weights))))
else:
    print("No feasible solution found.")

Quadratic program:
Problem name: Knapsack

Maximize
  3*x_0 + 4*x_1 + 5*x_2 + 6*x_3 + 7*x_4

Subject to
  Linear constraints (1)
    2*x_0 + 3*x_1 + 4*x_2 + 5*x_3 + 6*x_4 <= 10  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4


Using backend: <IBMBackend('ibm_brisbane')>


IBMInputValueError: 'The instruction ry on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'

In [11]:
from qiskit.circuit.library import EfficientSU2

num_qubits = op.num_qubits
print("Num qubits needed:", num_qubits)
print("Backend qubits:", backend.num_qubits)

if num_qubits > backend.num_qubits:
    raise ValueError("Problem requires more qubits than backend supports.")

# Use EfficientSU2 ansatz, which is more hardware-friendly
ansatz = EfficientSU2(num_qubits, reps=1, entanglement="linear")

Num qubits needed: 9
Backend qubits: 127


/var/folders/yr/gk8xm7h947qcpzxwlbcp54sm0000gn/T/ipykernel_98421/15291745.py:11: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits, reps=1, entanglement="linear")
